In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,521 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,904 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/un

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Code below modified from Module 22 class activities
spark.sparkContext.addFile(url)
homes_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True)

# Show DataFrame
homes_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT bedrooms, ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms = 4
GROUP BY bedrooms
""").show()

+--------+--------------------+
|bedrooms|round(avg(price), 2)|
+--------+--------------------+
|       4|           299661.01|
+--------+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT bedrooms, bathrooms, date_built, ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY bedrooms, bathrooms, date_built
ORDER BY date_built
""").show()

+--------+---------+----------+--------------------+
|bedrooms|bathrooms|date_built|round(avg(price), 2)|
+--------+---------+----------+--------------------+
|       3|        3|      2010|           292859.62|
|       3|        3|      2011|           291117.47|
|       3|        3|      2012|           293683.19|
|       3|        3|      2013|           295962.27|
|       3|        3|      2014|           290852.27|
|       3|        3|      2015|            288770.3|
|       3|        3|      2016|           290555.07|
|       3|        3|      2017|           292676.79|
+--------+---------+----------+--------------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT bedrooms, bathrooms, floors, date_built, ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY bedrooms, bathrooms, floors, date_built
ORDER BY date_built
""").show()

+--------+---------+------+----------+--------------------+
|bedrooms|bathrooms|floors|date_built|round(avg(price), 2)|
+--------+---------+------+----------+--------------------+
|       3|        3|     2|      2010|           285010.22|
|       3|        3|     2|      2011|           276553.81|
|       3|        3|     2|      2012|           307539.97|
|       3|        3|     2|      2013|           303676.79|
|       3|        3|     2|      2014|           298264.72|
|       3|        3|     2|      2015|           297609.97|
|       3|        3|     2|      2016|            293965.1|
|       3|        3|     2|      2017|           280317.58|
+--------+---------+------+----------+--------------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2)
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  52|           733780.26|
|  53|            755214.8|
|  54|           798684.82|
|  55|           771153.32|
|  56|            718176.4|
|  57|            734340.5|
|  58|           759764.65|
|  59|            791453.0|
|  60|           754939.65|
|  61|           746877.59|
|  62|           759150.14|
|  63|           711614.55|
|  64|           767036.67|
|  65|           736679.93|
|  66|            712475.0|
|  67|           737970.96|
|  68|           716785.44|
|  69|           750537.94|
|  70|           695865.58|
+----+--------------------+
only showing top 20 rows

--- 1.685492992401123 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2)
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  52|           733780.26|
|  53|            755214.8|
|  54|           798684.82|
|  55|           771153.32|
|  56|            718176.4|
|  57|            734340.5|
|  58|           759764.65|
|  59|            791453.0|
|  60|           754939.65|
|  61|           746877.59|
|  62|           759150.14|
|  63|           711614.55|
|  64|           767036.67|
|  65|           736679.93|
|  66|            712475.0|
|  67|           737970.96|
|  68|           716785.44|
|  69|           750537.94|
|  70|           695865.58|
+----+--------------------+
only showing top 20 rows

--- 0.7583377361297607 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

In [13]:
# 11. Read the parquet formatted data.
p_homes_df=spark.read.parquet('delayed_partitioned')

In [14]:
# 12. Create a temporary table for the parquet data.
p_homes_df.createOrReplaceTempView('p_home_sales')

In [15]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2)
FROM p_home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  52|           733780.26|
|  53|            755214.8|
|  54|           798684.82|
|  55|           771153.32|
|  56|            718176.4|
|  57|            734340.5|
|  58|           759764.65|
|  59|            791453.0|
|  60|           754939.65|
|  61|           746877.59|
|  62|           759150.14|
|  63|           711614.55|
|  64|           767036.67|
|  65|           736679.93|
|  66|            712475.0|
|  67|           737970.96|
|  68|           716785.44|
|  69|           750537.94|
|  70|           695865.58|
+----+--------------------+
only showing top 20 rows

--- 0.8470125198364258 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

True